[Reference](https://jadhav-pritish.medium.com/solving-markov-chains-using-python-35cc79efddd7)

In [13]:
import pandas as pd

# allow users to define a markov chain using a pandas dataframe

markov_chain_df = pd.DataFrame(
    {
        "from": ["A", "A", "B", "B"],
        "to": ["A", "B", "A", "B"],
        "weight": [0.7, 0.3, 0.2, 0.8],
    }
)

display(markov_chain_df)

[17/Nov/2022 13:40:53] INFO - NumExpr defaulting to 2 threads.


,from,to,weight
0,A,A,0.7
1,A,B,0.3
2,B,A,0.2
3,B,B,0.8


In [17]:
!pip install cached-property
!pip install networkx

from dataclasses import dataclass
import pandas as pd
import scipy
import networkx as nx
import numpy as np
from cached_property import cached_property

@dataclass(frozen=True)
class MarkovChain:

    edge_df: pd.DataFrame

    def __post_init__(self):
        assert all(self.edge_df.columns == ["from", "to", "weight"])

    @cached_property
    def markov_chain(self) -> nx.DiGraph:
        return nx.from_pandas_edgelist(
            self.edge_df, "from", "to", ["weight"], create_using=nx.DiGraph()
        )

    @cached_property
    def transition_matrix(self):
        return nx.adjacency_matrix(self.markov_chain).todense().T

    def is_markov_chain_valid(self):
        return nx.is_strongly_connected(self.markov_chain) and nx.is_aperiodic(
            self.markov_chain
        )

    def compute_steady_state_probability(self):

        assert (
            self.is_markov_chain_valid
        ), "The markov chain is inavlid. A valid Markov chain needs to be Irreducible and Aperiodic"

        # compute eigen values and eigen vectors for the transition matrix
        eigen_values, eigen_vectors = np.linalg.eig(self.transition_matrix)

        # steady state probability = eigen vector with eigen value 1
        eigen_val_index = [round(elem.real) for elem in eigen_values].index(1.0)

        # normalize the eigen vector
        return eigen_vectors[:, eigen_val_index] / sum(
            eigen_vectors[:, eigen_val_index]
        )
        
        
# allow users to define a markov chain using a pandas dataframe

markov_chain_df = pd.DataFrame(
    {
        "from": ["A", "A", "B", "B"],
        "to": ["A", "B", "A", "B"],
        "weight": [0.7, 0.3, 0.2, 0.8],
    }
)

display(markov_chain_df)

markov_chain_obj = MarkovChain(markov_chain_df)

# verify that the markov chain is valid
assert markov_chain_obj.is_markov_chain_valid()

# compute the steady state probabilities
print(markov_chain_obj.compute_steady_state_probability())

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


,from,to,weight
0,A,A,0.7
1,A,B,0.3
2,B,A,0.2
3,B,B,0.8


[[0.4]
 [0.6]]
